https://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page

In [2]:
import shapefile as shp
import math
import pandas as pd
import geopandas as gpd
import pylab as pl
from fiona.crs import from_epsg

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [10]:
hp_target = gpd.read_file("./hp_target/hp_target.shp")
hp_target.to_crs(epsg=2263, inplace=True)

In [5]:
ct = gpd.read_file("./nyct2010_17b/nyct2010.shp")
cb = gpd.read_file("./nycb2010_17b/nycb2010.shp")

https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/nyct2010_metadata.pdf?ver=17b

https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/nycb2010_metadata.pdf?ver=17b

In [9]:
ct.to_crs(epsg=2263, inplace=True)
cb.to_crs(epsg=2263, inplace=True)

In [18]:
ct.BoroCT2010.unique().shape
cb.BCTCB2010.unique().shape

(38794,)

## CT merge

#### intersect & within

In [39]:
hp_intersect_ct = gpd.sjoin(hp_target, ct, how='left', op='intersects')
hp_within_ct = gpd.sjoin(hp_target, ct, how='left', op='within')
ct_within_hp = gpd.sjoin(ct, hp_target, how='inner', op='within')

In [28]:
hp_intersect_ct.groupby('developmen').apply(lambda x: x.BoroCT2010)

developmen         
CHELSEA           0    1009300
CHELSEA ADDITION  1    1009700
ELLIOTT           2    1009700
                  2    1009300
FULTON            3    1008900
                  3    1008300
Name: BoroCT2010, dtype: object

In [35]:
hp_within_ct.groupby('developmen').apply(lambda x: x.BoroCT2010)

developmen         
CHELSEA           0    1009300
CHELSEA ADDITION  1    1009700
ELLIOTT           2        NaN
FULTON            3        NaN
Name: BoroCT2010, dtype: object

In [41]:
ct_within_hp.shape

(0, 15)

### CB merge

In [44]:
hp_intersect_cb = gpd.sjoin(hp_target, cb, how='left', op='intersects')
hp_within_cb = gpd.sjoin(hp_target, cb, how='left', op='within')
cb_within_hp = gpd.sjoin(cb, hp_target, how='inner', op='within')

In [37]:
hp_intersect_cb.groupby('developmen').apply(lambda x: x.BCTCB2010)

developmen         
CHELSEA           0    10093006000
                  0    10093006001
                  0    10093006002
CHELSEA ADDITION  1    10097002002
ELLIOTT           2    10093006001
                  2    10097002002
                  2    10093006002
FULTON            3    10083003000
                  3    10089003001
                  3    10089003000
                  3    10083003001
Name: BCTCB2010, dtype: object

In [38]:
hp_within_cb.groupby('developmen').apply(lambda x: x.BCTCB2010)

developmen         
CHELSEA           0            NaN
CHELSEA ADDITION  1    10097002002
ELLIOTT           2            NaN
FULTON            3            NaN
Name: BCTCB2010, dtype: object

In [45]:
cb_within_hp.shape

(0, 11)

<br/>
<br/>
### Select op = 'intersects'

In [49]:
hp_ct = hp_intersect_ct.groupby('developmen').apply(lambda x: x.BoroCT2010.values)
hp_cb = hp_intersect_cb.groupby('developmen').apply(lambda x: x.BCTCB2010.values)

In [55]:
hp_target['CT'] = hp_ct.values
hp_target['CB'] = hp_cb.values

In [56]:
hp_target

,borough,developmen,geometry,CT,CB
0,MANHATTAN,CHELSEA,"POLYGON ((983771.073124409 212170.9467828723, ...",[1009300],"[10093006000, 10093006001, 10093006002]"
1,MANHATTAN,CHELSEA ADDITION,"POLYGON ((983739.0203669845 212272.0843758562,...",[1009700],[10097002002]
2,MANHATTAN,ELLIOTT,(POLYGON ((983769.5173532362 212064.8298691196...,"[1009700, 1009300]","[10093006001, 10097002002, 10093006002]"
3,MANHATTAN,FULTON,(POLYGON ((982789.5691819044 210258.9745569509...,"[1008900, 1008300]","[10083003000, 10089003001, 10089003000, 100830..."
